<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/KNN_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KNN for CIFAR10 Classification (sklearn v1)

Dataset preparation

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=12):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

mean_cifar10, std_cifar10 = (0.5071, 0.4866, 0.4409), (0.2009, 0.1984, 0.2023)
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(), transforms.ToTensor(),
            transforms.Normalize(mean_cifar10, std_cifar10), ])
transform_test = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize(mean_cifar10, std_cifar10),])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2048, shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2048, shuffle=False, num_workers=2)

def get_data_processed(dataloader):
    with torch.no_grad():
        inputs_array_all = []
        targets_array_all = []
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs_array = np.array(inputs.view(inputs.shape[0], -1).cpu())
            targets_array = np.array(targets.cpu())
            inputs_array_all.extend(inputs_array)
            targets_array_all.extend(targets_array)

        return inputs_array_all, targets_array_all

img_data, img_label_orig = get_data_processed(train_loader)
test_img_data, test_label = get_data_processed(test_loader)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Fitting and Testing:

In [2]:
from sklearn.neighbors import KNeighborsClassifier
neighbors = KNeighborsClassifier(n_neighbors=3, algorithm='brute').fit(img_data, img_label_orig)
YPred_soa = neighbors.predict(test_img_data)
Accuracy = (test_label == YPred_soa).mean()
print('KNN Accuracy:', Accuracy)
for i in range (10):
    print('sample:',i, 'Label:',test_label[i], 'Pred:',YPred_soa[i])

KNN Accuracy: 0.2435
sample: 0 Label: 3 Pred: 4
sample: 1 Label: 8 Pred: 0
sample: 2 Label: 8 Pred: 0
sample: 3 Label: 0 Pred: 8
sample: 4 Label: 6 Pred: 4
sample: 5 Label: 6 Pred: 4
sample: 6 Label: 1 Pred: 2
sample: 7 Label: 6 Pred: 2
sample: 8 Label: 3 Pred: 4
sample: 9 Label: 1 Pred: 8


# KNN for CIFAR10 Classification (sklearn v2)

In [3]:
from sklearn.neighbors import KDTree

x_train, y_train, x_test, y_test = np.array(img_data), np.array(img_label_orig), np.array(test_img_data), np.array(test_label)
n_labels = np.max(y_train) + 1
kdtrees = [None] * n_labels

for label in range(n_labels):
    X_to_use = x_train[np.where(y_train == label)[0]]
    kdtrees[label] = KDTree(X_to_use)


k=2

In [16]:
d = np.tile(None, (x_test.shape[0], n_labels))
for label_idx in range(n_labels):
    d[:, label_idx] = kdtrees[label_idx].query(x_test, k=2)[0][:, -1]

pred = np.argmin(d, axis=1)
Accuracy = (test_label == pred).mean()
print('KNN Accuracy:', Accuracy)

KNN Accuracy: 0.2648


k=10

In [17]:
d = np.tile(None, (x_test.shape[0], n_labels))
for label_idx in range(n_labels):
    d[:, label_idx] = kdtrees[label_idx].query(x_test, k=10)[0][:, -1]

pred = np.argmin(d, axis=1)
Accuracy = (test_label == pred).mean()
print('KNN Accuracy:', Accuracy)


KNN Accuracy: 0.2658


# KNN for CIFAR10 Classification (scratch V1)

In [32]:
# K-NN Class
class NearestNeighbor(object):
  def __init__(self):
    pass

  def train(self, X, y):
    """ X is N x D where each row is an example. Y is 1-dimension of size N """
    # the nearest neighbor classifier simply remembers all the training data
    self.Xtr = X
    self.ytr = y

  def predict(self, X):
    """ X is N x D where each row is an example we wish to predict label for """
    num_test = X.shape[0]
    # lets make sure that the output type matches the input type
    Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

    # loop over all test rows
    for i in range(num_test):
      # find the nearest training image to the i'th test image
      # using the L1 distance (sum of absolute value differences)
      distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
      min_index = np.argmin(distances) # get the index with smallest distance
      Ypred[i] = self.ytr[min_index] # predict the label of the nearest example
      print("\rClassifiying {} ...".format(i),end="")

    return Ypred

print("LOADED CLASS ...")

# K-NN Classification

nn = NearestNeighbor() # create a Nearest Neighbor classifier class
nn.train(np.array(img_data), np.array(img_label_orig)) # train the classifier on the training images and labels
print("TRAINED CLASSIFIER ...")
Xt_predict = nn.predict(np.array(test_img_data))
Accuracy = (test_label == Xt_predict).mean()
print('KNN Accuracy:', Accuracy)

LOADED CLASS ...
TRAINED CLASSIFIER ...
Classifiying 8517 ...

KeyboardInterrupt: ignored

# KNN for CIFAR10 Classification (scratch V2)

In [2]:
class KNearestNeighbor(object):
    def __init__(self):
        pass
    def train(self, X, y):
        self.X_train = X
        self.y_train = y
    def predict(self, X, k=1, num_loops=0):
        if num_loops == 0:
            dists = self.compute_distances(X)
        else:
            raise ValueError('Invalid value %d for num_loops' % num_loops)
        return self.predict_labels(dists, k=k)


    def compute_distances(self, X):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train)) 
        dists = np.sqrt(np.sum(np.square(self.X_train), axis=1) + np.sum(np.square(X), axis=1)[:, np.newaxis] - 2 * np.dot(X, self.X_train.T))
        pass
        return dists

    def predict_labels(self, dists, k=1):
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            closest_y = []
            sorted_dist = np.argsort(dists[i])
            closest_y = list(self.y_train[sorted_dist[0:k]])
            pass
            y_pred[i]= (np.argmax(np.bincount(closest_y)))
            pass
        return y_pred

classifier = KNearestNeighbor()
classifier.train(np.array(img_data), np.array(img_label_orig))
dists= classifier.compute_distances(np.array(test_img_data))
y_test_pred = classifier.predict_labels(dists, k=5)
accuracy = np.mean(y_test_pred == test_label)
print('accuracy: %f' % (accuracy))

accuracy: 0.262000


# KNN for CIFAR10 Classification (scratch V3)[crashed]

In [ ]:
def compute_distances_no_loops(x_train, x_test):
      """
      Inputs:
      x_train: shape (num_train, C, H, W) tensor.
      x_test: shape (num_test, C, H, W) tensor.

      Returns:
      dists: shape (num_train, num_test) tensor where dists[i, j] is the
         Euclidean distance between the ith training image and the jth test
         image.
      """
      # Get number of training and testing images
      num_train = x_train.shape[0]
      num_test = x_test.shape[0]

      # Create return tensor with desired dimensions
      dists = x_train.new_zeros(num_train, num_test) # (500, 250)

      # Flattening tensors
      train = x_train.flatten(1) # (500, 3072)
      test = x_test.flatten(1) # (250, 3072)

      # Find Euclidean distance
      # Squaring elements
      train_sq = torch.square(train)
      test_sq = torch.square(test)

      # Summing row elements
      train_sum_sq = torch.sum(train_sq, 1) # (500)
      test_sum_sq = torch.sum(test_sq, 1) # (250)

      # Matrix multiplying train tensor with the transpose of test tensor
      mul = torch.matmul(train, test.transpose(0, 1)) # (500, 250)

      # Reshape enables proper broadcasting.
      # train_sum_sq = [500, 1] shape tensor and test_sum_sq = [1, 250] shape tensor.
      # This enables broadcasting to match desired dimensions of dists
      dists = torch.sqrt(train_sum_sq.reshape(-1, 1) + test_sum_sq.reshape(1, -1) - 2*mul)

      return dists

class KnnClassifier:
    def __init__(self, x_train, y_train):
        """
        x_train: shape (num_train, C, H, W) tensor where num_train is batch size,
        C is channel size, H is height, and W is width.
        y_train: shape (num_train) tensor where num_train is batch size providing labels
        """

        self.x_train = x_train
        self.y_train = y_train

    def predict(self, x_test, k=1):
        """
        x_test: shape (num_test, C, H, W) tensor where num_test is batch size,
        C is channel size, H is height, and W is width.
        k: The number of neighbors to use for prediction
        """

        # Init output shape
        y_test_pred = torch.zeros(x_test.shape[0], dtype=torch.int64)

        # Find & store Euclidean distance between test & train
        dists = compute_distances_no_loops(self.x_train, x_test)

        # Index over test images
        for i in range(dists.shape[1]):
            # Find index of k lowest values
            x = torch.topk(dists[:,i], k, largest=False).indices

            # Index the labels according to x
            k_lowest_labels = self.y_train[x]

            # y_test_pred[i] = the most frequent occuring index
            y_test_pred[i] = torch.argmax(torch.bincount(k_lowest_labels))
        
        return y_test_pred

    def check_accuracy(self, x_test, y_test, k=1, quiet=False):
        """
        x_test: shape (num_test, C, H, W) tensor where num_test is batch size,
        C is channel size, H is height, and W is width.
        y_test: shape (num_test) tensor where num_test is batch size providing labels
        k: The number of neighbors to use for prediction
        quiet: If True, don't print a message.

        Returns:
        accuracy: Accuracy of this classifier on the test data, as a percent.
        Python float in the range [0, 100]
        """

        y_test_pred = self.predict(x_test, k=k)
        num_samples = x_test.shape[0]
        num_correct = (y_test == y_test_pred).sum().item()
        accuracy = 100.0 * num_correct / num_samples
        msg = (f'Got {num_correct} / {num_samples} correct; '
            f'accuracy is {accuracy:.2f}%')
        if not quiet:
            print(msg)
        return accuracy

torch.manual_seed(0)
num_train = 5000
num_test = 500
#x_train, y_train, x_test, y_test = np.array(img_data), np.array(img_label_orig), np.array(test_img_data), np.array(test_label)
x_train, y_train, x_test, y_test = torch.tensor(np.array(img_data)), torch.tensor(np.array(img_label_orig)), torch.tensor(np.array(test_img_data)), torch.tensor(np.array(test_label))
#x_train, y_train, x_test, y_test = cifar10(num_train, num_test)

classifier = KnnClassifier(x_train, y_train)
classifier.check_accuracy(x_test, y_test, k=5)

# KNN for CIFAR10 Classification (scratch V4)

In [2]:
import numpy as np
import pickle
import os
#from settings import PROJECT_ROOT
import sys
import platform
class NearestNeighbor(object):
  def __init__(self):
    pass

  def train(self, X, y):
    """ X is N x D where each row is an example. Y is 1-dimension of size N """
    # the nearest neighbor classifier simply remembers all the training data
    self.Xtr = X
    self.ytr = y

  def predict(self, X):
    """ X is N x D where each row is an example we wish to predict label for """
    num_test = X.shape[0]

    print(num_test)
    # lets make sure that the output type matches the input type
    Ypred = np.zeros(num_test, dtype = self.ytr.dtype)
    xrange=range
    # loop over all test rows
    for i in xrange(num_test):
      print(i)
      # find the nearest training image to the i'th test image
      # using the L1 distance (sum of absolute value differences)
      distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
      min_index = np.argmin(distances) # get the index with smallest distance
      Ypred[i] = self.ytr[min_index] # predict the label of the nearest example
    return Ypred  
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,3):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte  



Xtr, Ytr, Xte, Yte = load_CIFAR10('data/cifar-10-batches-py/') # a magic function we provide
# flatten out all images to be one-dimensional
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) # Xtr_rows becomes 50000 x 3072
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3) # Xte_rows becomes 10000 x 3072

nn = NearestNeighbor() # create a Nearest Neighbor classifier class
nn.train(Xtr_rows, Ytr) # train the classifier on the training images and labels
Yte_predict = nn.predict(Xte_rows) # predict labels on the test images
# and now print the classification accuracy, which is the average number
# of examples that are correctly predicted (i.e. label matches)
print ('accuracy: %f' % ( np.mean(Yte_predict == Yte) )) # accuracy: 0.338500

Streaming output truncated to the last 5000 lines.
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190